In [1]:
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import PIL
from skimage.measure import compare_psnr, compare_ssim

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

In [12]:
def visualize_sample(lr_files, target_files, files, sample):
    targ_im = PIL.Image.open(target_files[sample])
    lr_im = PIL.Image.open(lr_files[sample])
    targ_arr = np.asarray(targ_im, np.uint8)
    lr_arr = np.asarray(lr_im, np.uint8)
    
    n_models = len(files)
    n_rows = n_models//2 + n_models%2 + 1
    model_im = [PIL.Image.open(file_list[sample]) for file_list in files]
    model_arr = [np.asarray(im, np.uint8) for im in model_im]
    psnr = [compare_psnr(targ_arr, arr, data_range=255.) for arr in model_arr]
    ssim = [compare_ssim(targ_arr, arr, data_range=255.) for arr in model_arr]
    
    f, axarr = plt.subplots(n_rows, 2) # create visualizations
    if n_models%2:
        f.delaxes(axarr[n_rows-1,1])
    axarr[0][0].imshow(np.squeeze(lr_arr), cmap=plt.cm.gray) # visualize original image file
    axarr[0][0].set_title('LR')
    axarr[0][1].imshow(np.squeeze(targ_arr), cmap=plt.cm.gray) # visualize original image file
    axarr[0][1].set_title('HR')
    for i in range(n_models):
        axarr[(i+2)//2][(i+2)%2].imshow(np.squeeze(model_arr[i]), cmap=plt.cm.gray) # visualize image tensor
        axarr[(i+2)//2][(i+2)%2].set_title(titles[i])
        axarr[(i+2)//2][(i+2)%2].set_xlabel(f"PSNR: {psnr[i]:.2f}, SSIM: {ssim[i]:.2f}")
    plt.show() # show visualization

# Load Data

In [4]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
infer_pth = data_pth/f'EM/testing/self_sv/'

In [5]:
model_list = {
    'baseline' : 'baseline/5.16_mse_baseline_3.pkl',
    'self_critic' : 'selfcritic-pretrained-best',
    'inpaint' : 'inpainting/5.4_superres_transfer_3.pkl',
    'contrastive' : 'contrastive/clr-finetune',
    'imagenet_critic' : 'inpainting/5.27_imagenet_critic_3.pkl',
}

In [6]:
titles = list(model_list.keys())

# Visualization

In [7]:
dirs = [f'{model_name}/real-world_SEM/' for model_name in model_list.values()]
paths = [infer_pth/dir1 for dir1 in dirs]
files = [sorted(list(pth1.glob('*.tif'))) for pth1 in paths]

lr_dirname = data_pth/f'EM/testing/LR/real-world_SEM/'       # target images
lr_files = sorted(list(lr_dirname.glob('*.tif')))
target_dirname = data_pth/f'EM/testing/HR/real-world_SEM/'       # target images
target_files = sorted(list(target_dirname.glob('*.tif')))
print('Processing '+str(len(target_files))+' files...')

Processing 42 files...


In [8]:
# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 25
fig_size[1] = 48
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


In [13]:
@interact
def show_sample(sample=33):
    return visualize_sample(lr_files, target_files, files, sample)

interactive(children=(IntSlider(value=33, description='sample', max=99, min=-33), Output()), _dom_classes=('wi…